# 주제 : 뉴스기사 생성 모델 구현하기

이번 튜토리얼에서는 LSTM layer를 활용하여 가짜 뉴스기사 생성기를 만들어 보도록 하겠습니다. 튜토리얼을 진행하면서 **한글 자연어 전처리, 학습을 위한 데이터셋 구축, 그리고 LSTM 텍스트 생성기 모델** 완성을 학습하실 수 있습니다.

## Step 1. 데이터 불러오기 및 전처리

### 문제 01. 필요한 모듈 import

In [ ]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd
import os
import re

### 문제 02. 데이터 불러오기

- 한글 뉴스기사 데이터셋을 받습니다.
- 한글 뉴스기사 200개의 데이터셋입니다.
- IT 관련 기사 데이터셋입니다. IT 관련 뉴스기사를 학습 시키므로, 향후 예측시 IT 관련된 뉴스기사가 생성됩니다.

In [ ]:
df = pd.read_csv('https://bit.ly/3n7iHQX')

- 전처리를 진행합니다.
- re 모듈을 사용하며 regular expression 문법을 사용하여, 한글, 영어, 숫자를 제외한 모든 문자는 제거합니다.
- 문장의 끝에는 '#'를 추가하여, 신문 기사의 끝이라는 표기를 해줍니다.

In [ ]:
def clean_sentence(sentence):
    # 한글, 영어, 숫자를 제외한 모든 문자는 제거합니다.
    sentence = re.sub(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    # 문장의 끝을 표기합니다.
    sentence += ' #'
    return sentence

In [ ]:
clean_sentence('abcef가나다^^$%@12시 땡^^!??')

데이터프레임의 `text`에 `clean_sentece`를 적용합니다.

In [ ]:
# 코드를 입력하세요
df['text'] = 

In [ ]:
df.head()

### 문제 03. 데이터 프레임에서 text만 병합하기

`text` 변수에 데이터프레임의 담긴 모든 기사를 join하여 병합합니다.

데이터 프레임의 `text`를 모두 병합합니다.

In [ ]:
# 코드를 입력하세요


총 문장의 길이는 다음과 같습니다.

In [ ]:
# 총 문장의 길이
len(text)

문장의 500 글자만 출력해 봅니다. 전처리가 완료된 문장이 출려됨을 확인할 수 있습니다.

In [ ]:
print(text[:500])

### 문제 04. 텍스트 기본 전처리 (preprocessing)

단어 사전을 만듭니다. 먼저, 중복되는 모든 글자를 제외하기 위하여 **set를 활용**합니다.

In [ ]:
# 코드를 입력하세요
vocab = 

In [ ]:
vocab[:20]

고유 글자의 숫자를 확인합니다.

In [ ]:
# 고유 글자의 숫자 확인
len(vocab)

'?'라는 글자는 `vocab` 변수에 없는 글자임을 확인할 수 있습니다. 전처리 단계에서 제거했기 때문에 당연히 없습니다.

In [ ]:
'?' in vocab

**'?'** 글자를 추가해 주는데, 추후 사용자의 입력이 없는 글자일 때는 ?로 입력하기 위함입니다.

In [ ]:
# 코드를 입력하세요
vocab.

### 문제 05. 데이터 형태 변환하기

`char2idx`는 글자를 index로 변환하는 역할이고, `idx2char`는 index를 글자로 역변환하는 목적입니다.

In [ ]:
# 글자 -> index로 변환
# 코드를 입력하세요
char2idx = 

In [ ]:
# index -> 글자로 변환
# 코드를 입력하세요
idx2char = 

## STEP 2. 단어 사전 만들기

### 문제 06. for문을 사용해 문서를 연속된 수치형 값들로 치환합니다.

문장을 `char2idx`를 활용하여 텍스트를 int로 변환합니다. (sequence 변환)

In [ ]:
# 코드를 입력하세요
text_as_int = 

In [ ]:
text_as_int

In [ ]:
len(text_as_int)

### 문제 07. 변환된 부분을 확인합니다. (처음 5개)

**원문의 출력**은 다음과 같습니다.

In [ ]:
# 원문
text[:5]

**sequence로 변환**된 출력은 다음과 같습니다. 한글자씩 변환된 것을 볼 수 있습니다.

In [ ]:
char2idx['갤'], char2idx['럭'], char2idx['시'], char2idx['S'], char2idx['9']

In [ ]:
# 변환된 sequence
text_as_int[:5]

### 문제 08. 각각의 단어사전으로 출력합니다.

In [ ]:
char2idx[' '], char2idx['회'], char2idx['사'], char2idx['#'], char2idx['?'], 

(0, 1144, 599, 1, 1172)

## Step 3. 데이터셋 생성 및 EDA

### 문제 09. X, Y 데이터셋 생성하기

In [ ]:
# 아래 코드는 그대로 실행해주세요.
# 단일 입력에 대해 원하는 문장의 최대 길이를 지정합니다.
window_size = 100
shuffle_buffer = 1000
batch_size=128

In [ ]:
# 데이터셋을 만드는 함수를 구현해봅니다.
# 코드를 입력하세요
def windowed_dataset(series, window_size, shuffle_buffer, batch_size):
    series = tf.expand_dims(series, -1)
    # numpy array를 tensor로 변환
    series = tf.expand_dims(series, -1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    # window 구성 (shift=1, drop_remainder=True 설정)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    # flat_map 적용
    ds = ds.flat_map(lambda x: x.batch(window_size + 1))
    # shuffle 적용
    ds = ds.shuffle(shuffle_buffer)
    # x, y 분할
    return ds.batch(batch_size).prefetch(1).repeat()

In [ ]:
train_data = windowed_dataset(np.array(text_as_int), window_size, shuffle_buffer, batch_size)

### 문제 10. 어휘 사전의 크기를 간단히 살펴봅니다.

In [ ]:
# 문자로 된 어휘 사전의 크기
# vocab_size에 vocab의 길이를 대입합니다.
# 코드를 입력하세요
vocab_size = 
vocab_size

## Step 4.Sequential 모델 구현하기

### 문제 11. keras를 활용해 Sequential 모델을 구현합니다.

hyperparameter를 다음과 같이 설정합니다. 데이터셋에 따라서 언제든 변경하면서 더 좋은 성능을 내는 hyperparameter 값을 찾을 수 있습니다.

In [ ]:
# 아래 코드는 그대로 실행해주세요.
# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [ ]:
model = tf.keras.Sequential([
    # Embedding Layer
    
    # LSTM Layer (returen_sequences=True, initializer는 glorot_uniform으로 설정)
    
    # Dense의 unit은 vocab_size로 설정
    
])

In [ ]:
model.summary()

### 문제 12. 모델을 저장할 Checkpoint를 생성합니다.

In [ ]:
checkpoint_path = 'sample-checkpoint.h5'
# 코드를 입력하세요
checkpointer = tf.keras.callbacks.ModelCheckpoint(
    filepath=
    save_best_only=
    monitor=
    verbose=1, 
)

### 문제 14. 모델을 컴파일합니다. 옵티마이저는 adam을 사용해주세요.

In [ ]:
model.compile(optimizer=
              loss=
              metrics=
              )

### 문제 15. steps_per_epoch를 지정합니다.

**steps_per_epoch에 대하여**


1. fit()함수를 취할 때, 버젼별로 `steps_per_epoch`과, `validation_steps`의 값이 지정되어 있지 않으면 학습이 안되는 현상이 있습니다.

2. 따라서, 위의 2가지 파라미터에 값을 넣어 주면 정상적으로 학습하기 시작합니다.

3. `steps_per_epoch`은 weight를 업데이트 하는 주기 입니다. 보통은 이미지 데이터셋의 총 갯수가 1000개 일 때, `batch_size`가 128개 이면, **이미지갯수**/**배치사이즈**만큼 weight를 업데이트 합니다. 쉽게 말해서, batch가 다 돌때마다 weight를 업데이트 합니다.

4. 1000개 이미지 / 128 하면 7.8125 (소수점)이 나오기 때문에 1000 // 128 해주면 해결되는데요. 몫을 구하면 7이나오기 때문에 +1을 해주면 됩니다. 즉, 마지막 batch는 128개보다 적겠네요. 그래도 weight 업데이트 해줘야하니깐 최종 `steps_per_epoch`은 8이 맞습니다.

5. validation_steps는 validation_generator의 weight 업데이트 숫자입니다. 4번에서 구한 `steps_per_epoch`과 동일한 원리로 validation dataset을 기준으로 계산하면 됩니다.

6. validation data는 500개고, batch_size가 이번에는 32개면 500 // 32 + 1 = 16 이 되겠네요.

7. 하지만, 이런 고민 모두다 필요없이 `steps_per_epoch` = len(training_generator)
`validation_steps` = len(validation_generator) 로 설정해주면 초 간단합니다^^

In [ ]:
steps_per_epoch = (len(text_as_int) - window_size) // (batch_size)
steps_per_epoch

### 문제 16. 모델을 학습하고 callbacks로 앞에서 만든 체크포인트를 할당해줍니다.

In [ ]:
# 이어서 학습시 아래 코드를 실행합니다.
# model.load_weights(checkpoint_path)

In [ ]:
model.fit(train_data, 
          epochs= 
          steps_per_epoch=
          callbacks=
          )

## Step 5. 모델을 활용한 뉴스기사 생성

In [ ]:
model = tf.keras.Sequential([
    # Embedding Layer
    
    # LSTM Layer (returen_sequences=True, initializer는 glorot_uniform으로 설정)
    
    # Dense의 unit은 vocab_size로 설정
    
])

### 문제 17. 저장한 Model Checkpoint를 불러옵니다.

In [ ]:
# 코드를 입력하세요


### 문제 18. 모델을 build하고 요약 내용을 출력해봅니다.

In [ ]:
model.

In [ ]:
model.summary()

### 문제 19. 불러온 모델을 활용해 뉴스기사를 생성해봅니다.

In [ ]:
def generate_text(model, start_string):
    # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

    # 생성할 문자의 수
    num_generate = 1000

    # 시작 문자열을 숫자로 변환(벡터화)
    input_eval = 
    input_eval = tf.expand_dims(input_eval, 0)

    # 결과를 저장할 빈 문자열
    text_generated = []

    # 온도가 낮으면 더 예측 가능한 텍스트가 됩니다.
    # 온도가 높으면 더 의외의 텍스트가 됩니다.
    # 최적의 세팅을 찾기 위한 실험
    temperature = 0.1

    # 여기에서 배치 크기 == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = 
        # 배치 차원 제거
        predictions = 

        # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
        predictions = 
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # 예측된 단어를 다음 입력으로 모델에 전달
        # 이전 은닉 상태와 함께
        input_eval = tf.expand_dims([predicted_id], 0)
        result_char = idx2char[predicted_id]
        
        # '#' 문자열을 만나면 종료합니다.
        if result_char == '#':
            break
        
        text_generated.append(result_char)

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"스마트폰 "))